Principal Component Analysis (PCA) is a linear transformation that transforms data points into feature space representation. It has the following properties:

* Exactly the same dimension as the original data space
* Reconstruction of data points is optimal in the mean square error sense
* Principal components are sorted by "effectiveness" at capturing variance
* Dimensionality reduction can be done simply by truncating principal components

http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

In [ ]:
datasource = "../datasets/winequality-red.csv"
print(os.path.exists(datasource))

In [ ]:
df = pd.read_csv(datasource).sample(frac = 1).reset_index(drop = True)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
X = np.array(df.iloc[:, :-1]) # everything except for quality

In [ ]:
y = np.array(df["quality"]) # just the quality column

## Principal component analysis with sklearn
PCA transforms the data space into feature space. It is done by a matrix transform followed by truncation of low variance principal components. Then you are free to use the obtained features X_features as input to your model

In [ ]:
pca = PCA(n_components = 5)

In [ ]:
pca.fit(X)

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
X_features = pca.transform(X)

In [ ]:
print(X_features)

In [ ]:
print("Features Shape:\n", X_features.shape)

## Reconstruction of data space
An approximation of the original dataset can be recovered by linearly combining high variance principal components. This demonstrates the connection between data space and feature space. The difference between X_synthesized and X_reconstructed is that the former is centered at the origin because PCA assumes mathematical expectation of input to be zero and input needs to be centered otherwise. Therefore, in order to reconstruct the original dataset, the approxmiation obtained using PCA has to be shifted back.

In [ ]:
print("Principal components shape:", pca.components_.shape)

In [ ]:
X_synthesized = np.dot(X_features, pca.components_)

In [ ]:
X_reconstructed = X_synthesized + np.mean(X, axis = 0)[np.newaxis, ...]

In [ ]:
print("Reconstructed dataset shape:", X_reconstructed.shape)

In [ ]:
# reconstructed dataset, an approximation of original dataset
pd.DataFrame(X_reconstructed, columns = df.columns[:-1])[0:5]

In [ ]:
df.iloc[0:5, :-1]

Compared to the original dataset

## Approximation error of principal components
As a result, we're able to represent the original dataset using principal components without losing too much information. Space consumption was essentially halved. And these principal components can be used as features for subsequent machine learning pipelines. The following calculates mean squared error between the original df and the reconstructed dataset for all the cells

In [ ]:
error = X - X_reconstructed

In [ ]:
np.mean((error**2))

## Verify the eigenstructure of PCA

The solution of PCA could be found by performing eigen-decomposition of the covariance matrix of X. This section quickly verifies some of its properties pertaining to the eigenstructure. 

Some other interesting facts about PCA include:
* The error and approximation are othogonal. Because approximation consists of high variance principal components while the error solely consists of low variance principal components, and all principal components are othogonal pairwise. 

## Scree plot
This plots variance (y axis) against components (x axis). As one moves to the right, toward later components, the variances (or the eigenvalues) drop. It helps us decide on the number of principal components to be retained.

In [ ]:
x_ticks = np.arange(len(pca.components_)) + 1
plt.xticks(x_ticks) # this enforces integers on the x axis
plt.plot(x_ticks, pca.explained_variance_)

You can also plot the explained variance ratio. A full PCA without components truncated would have total explained variance ratio equal to 1.

In [ ]:
plt.xticks(x_ticks)
plt.plot(x_ticks, pca.explained_variance_ratio_)
print("Total Explained Variance Ratio\n", np.sum(pca.explained_variance_ratio_))